In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics.scorer import make_scorer

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
from preprocess import preprocess_train, preprocess, FEATURES, CATEGORICAL_FEATURES, TEST_FEATURES, CATEGORICAL_TEST_FEATURES_IDX

In [4]:
#train[u'Входные группы']

In [5]:
X, y = preprocess_train(train, categotical_features=CATEGORICAL_FEATURES, features=TEST_FEATURES)

In [6]:
X, y = shuffle(X, y)

In [7]:
y.shape

(8716,)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75)

In [16]:
df = pd.DataFrame(y_train)
df[df < 100] = 6666666
df

,0
0,288.90
1,222.40
2,203.27
3,460.70
4,397.30
5,514.80
6,6666666.00
7,599.00
8,400.40
9,535.57


## CatBoost

#### CrossVal mean scores
* catboost_d10: 224.01548277848525
* catboost_d16: 

In [10]:
# Training simple model
from catboost import CatBoostRegressor

In [11]:
cbr = CatBoostRegressor(iterations=10000, logging_level='Silent', depth=10, task_type='GPU')

In [ ]:
scorer = make_scorer(lambda a, b: mean_squared_error(a, b)**.5)
scores = cross_val_score(cbr, X, y, cv=5, scoring=scorer, verbose=1)

In [12]:
#np.mean(scores)

NameError: name 'scores' is not defined

In [36]:
y_pred = cbr.fit(X_train, y_train, cat_features=CATEGORICAL_TEST_FEATURES_IDX).predict(X_test)
print "Mean squared error: {}".format(mean_squared_error(y_test, y_pred)**.5)

Mean squared error: 284.83019248


In [13]:
# Make submission
cbr.fit(X, y, cat_features=CATEGORICAL_TEST_FEATURES_IDX)

In [14]:
test = preprocess(test, CATEGORICAL_FEATURES)

In [15]:
X = test[TEST_FEATURES].values

In [16]:
predictions = cbr.predict(X)

In [17]:
predictions

array([ 879.78794459, 1165.70020084,  279.39788828, ...,  154.47198918,
        -62.34725293,   36.55400176])

In [18]:
def make_submission(ids, predictions):
    df = pd.concat([ids, pd.Series(predictions)], axis=1)
    return df.rename(columns={0: 'value'})

In [19]:
l = test.id

In [20]:
df = pd.concat([test.id, pd.Series(predictions)], axis=1)

In [21]:
df = df.rename(columns={0: 'value'})

In [22]:
df.to_csv("catboost_d10.csv", index=False)